<a href="https://colab.research.google.com/github/MJMortensonWarwick/large_scale_data_for_research/blob/main/relational_dbs_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DuckDB in Python
Although there are many database options for your (Python) project, in recent years DuckDB has become particularly popular. There are several reasons for this, not least that DuckDB is very lightweight and easy to integrate. In fact, we don't need to worry about command-line installation, setting-up admin accounts or any of these other common tasks. While at an organisational level we may want these security measures and control, to quickly get something working in a classroom or for a "hack-style" app, DuckDB is particularly attractive. Let's get started:

In [3]:
# If on your own machine you will probably need to !pip install duckdb
import duckdb

# Create a new database and connect with read and write options
# content is the folder we work in for Colab notebooks - change as required if
# running on a local machine
conn = duckdb.connect('/content/mydb.db', read_only=False)

Here we have imported the package and created an active connection. By specifying a file name as the connection we have created local storage for our database. DuckDB does also allow for "in-memory" mode where our DB is purely in RAM and destroyed at the end of our session. We also specify our connection is not "read-only", meaning we can write to our database.

Let's quickly check the DB has been saved to disk:

In [4]:
import os
with os.scandir('/content') as entries:
    for entry in entries:
        print(entry.name)

.config
mydb.db.wal
mydb.db
sample_data


Everything has worked!

To start building a database. We will first build the _orders_ table we worked with previously. We probably want to include a primary key ("id") to distinguish records. Typically this is an auto-increment integer (first record "id = 1", second record "id = 2" and so on. These work slightly differently in DuckDB and we need to first create a _sequence_:

In [5]:
conn.execute("CREATE SEQUENCE id_seq START 1;")

Firstly, to explain something of the syntax. We are using the variable _conn_ that we created earlier when specifying our DB. This is, as the name suggests, a connection to this database which allows us to directly _execute_ SQL commands (written as Python strings). Note we emulate common SQL patterns of capitalisation and end the command with semi-colon (";"). Note also as well our SQL should always be wrapped in quotation marks (i.e. "_SQL\_COMMAND_") and anything within the command should use punctuation marks (e.g. "_SELECT * FROM table WHERE name = 'Jardon';_").

Secondly, let's discuss the specific command. We have created a sequence named _id\_seq_ which starts at "1". This means our first element of the sequence will be 1, the next 2 and so on. With this in place we can create a table:

In [6]:
conn.execute('''
CREATE TABLE orders(
  id INTEGER NOT NULL PRIMARY KEY default NEXTVAL('id_seq'),
  trans_date VARCHAR default NULL,
  product_id INTEGER default NULL,
  category_id INTEGER default NULL,
  price VARCHAR default NULL,
  county VARCHAR default NULL,
  customer_surname VARCHAR default NULL,
  customer_firstname VARCHAR default NULL,
  date_of_birth VARCHAR,
  newsletter VARCHAR default NULL,
  promo_code VARCHAR default NULL
  );
''')

As said, very similar. We list a table name and a sequence of fields. For each field we specify a data type, although we simplify by just giving a generic type rather than a size (DuckDB will infer the rest). We can also specify our default value (_NULL_ in most cases), whether _NULL_ is allowed and we can include various other 'rules'.

The one field which is slightly different is our _id_ field. Here we use our _id\_seq_ sequence to manage auto-incrementation as the default value and we specify this as the primary key at the point of definition rather than at the end of the _CREATE TABLE_ command.

With the table setup we can add data, again in a similar way as before:

In [7]:
conn.execute("INSERT INTO orders VALUES (NEXTVAL('id_seq'),'2019-08-17',56,2,'73.92','Shetland','Bravo','Sid','1966-11-01','1','SUMMER MADNESS');")

Here we have added a single row by passing table name and the sequence of values as a tuple. We should note we pass the _id_ first of all (using our _id\_seq_ sequence) and all string/VARCHAR values with punctuation marks (' ').

We can check this has worked by running a query:

In [8]:
conn.execute("SELECT * FROM orders;").fetchall()

[(1,
  '2019-08-17',
  56,
  2,
  '73.92',
  'Shetland',
  'Bravo',
  'Sid',
  '1966-11-01',
  '1',
  'SUMMER MADNESS')]

Our SQL syntax is identical as before, but note we need to pass the Python command _fetchall()_ in order to bring the results into our Python session and print these to screen.

Everything seems to be working so let's add the rest of the data:

In [9]:
cmdOne = "INSERT INTO 'orders' VALUES (NEXTVAL('id_seq'),'2019-07-04',43,2,'70.59','Cardiganshire','Patrick','Elvis','1974-01-07','1','PROMO10'),(NEXTVAL('id_seq'),'2019-07-07',22,2,'24.31','Suffolk','Davenport','Tyrone','1989-03-16','1',''),(NEXTVAL('id_seq'),'2019-01-26',1,1,'46.51','Inverness-shire','Jensen','Jillian','1948-05-07','0','PROMO15'),(NEXTVAL('id_seq'),'2019-02-07',11,2,'83.10','Lincolnshire','Sexton','Mona','1975-05-01','0',''),(NEXTVAL('id_seq'),'2019-09-02',90,3,'99.12','Ross-shire','Hooper','Xandra','1975-01-22','0',''),(NEXTVAL('id_seq'),'2019-09-19',89,3,'98.06','Midlothian','Leonard','Velma','1970-12-05','0','PROMO10'),(NEXTVAL('id_seq'),'2019-04-24',75,2,'62.08','Cambridgeshire','Pope','Rana','1954-07-30','0',''),(NEXTVAL('id_seq'),'2019-11-01',66,3,'63.58','Caithness','Velez','Griffin','1938-08-05','0','PROMO15'),(NEXTVAL('id_seq'),'2019-09-20',99,5,'63.79','Surrey','Anderson','Hadassah','1935-05-16','1','PROMO10');"
cmdTwo = "INSERT INTO 'orders' VALUES (NEXTVAL('id_seq'),'2019-01-27',16,4,'61.73','Flintshire','Barr','Quinn','1942-09-15','1','PROMO10'),(NEXTVAL('id_seq'),'2019-03-26',65,3,'16.36','Berwickshire','Carr','Grace','1934-08-10','0',''),(NEXTVAL('id_seq'),'2019-07-18',61,4,'53.26','Derbyshire','Dillon','Amethyst','1961-06-13','1',''),(NEXTVAL('id_seq'),'2019-01-29',62,5,'87.26','Buckinghamshire','Lee','Fritz','1943-09-02','1','PROMO10'),(NEXTVAL('id_seq'),'2019-02-24',93,4,'77.27','Caithness','Hunt','Caesar','1962-07-20','1',''),(NEXTVAL('id_seq'),'2019-05-16',32,3,'76.93','Denbighshire','Cleveland','Amethyst','1999-03-31','1',''),(NEXTVAL('id_seq'),'2019-07-16',11,2,'59.38','Carmarthenshire','Greene','Clare','1952-06-21','1','PROMO15'),(NEXTVAL('id_seq'),'2019-08-19',44,5,'21.04','Dunbartonshire','Branch','Brenda','1983-03-10','1',''),(NEXTVAL('id_seq'),'2019-10-29',44,1,'91.95','Wiltshire','Lang','Hunter','1981-12-10','1',''),(NEXTVAL('id_seq'),'2019-02-18',62,1,'56.23','Caithness','Mueller','Kyla','1990-04-21','0','PROMO10');"
cmdThree = "INSERT INTO 'orders' VALUES (NEXTVAL('id_seq'),'2019-09-25',33,2,'32.98','Somerset','Brown','Dahlia','1990-09-20','1','PROMO15'),(NEXTVAL('id_seq'),'2019-12-11',93,5,'43.40','Suffolk','Davenport','Tyrone','1989-03-16','1','PROMO10'),(NEXTVAL('id_seq'),'2019-11-22',70,4,'39.67','Cambridgeshire','Bruno','Jordan','1997-06-05','0','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-07-11',78,3,'45.14','Norfolk','Schneider','Thor','1942-05-28','1',''),(NEXTVAL('id_seq'),'2019-09-24',13,1,'80.68','Angus','Simpson','Nicole','1973-07-11','0',''),(NEXTVAL('id_seq'),'2019-11-11',13,3,'49.72','Suffolk','Golden','Martin','1974-05-26','0',''),(NEXTVAL('id_seq'),'2019-10-20',1,4,'14.16','Hampshire','Massey','Fiona','1975-02-24','0','PROMO10'),(NEXTVAL('id_seq'),'2019-03-02',50,4,'28.68','Cumberland','Jefferson','Ashton','1976-12-03','1',''),(NEXTVAL('id_seq'),'2019-05-08',12,2,'60.25','Wigtownshire','Ingram','Hammett','1956-10-21','1',''),(NEXTVAL('id_seq'),'2019-06-22',17,4,'50.22','Denbighshire','Dickerson','Evangeline','1972-08-16','0','PROMO15');"
cmdFour = "INSERT INTO 'orders' VALUES (NEXTVAL('id_seq'),'2019-12-02',5,5,'11.52','Westmorland','Sosa','Armand','1975-06-11','0','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-08-14',37,3,'95.56','Bedfordshire','Warren','Nicole','2004-03-28','1','PROMO15'),(NEXTVAL('id_seq'),'2019-02-05',89,2,'3.73','Selkirkshire','Sparks','Dolan','1952-07-14','1','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-05-21',2,1,'64.61','Caithness','Bowman','Naomi','1979-04-16','1','PROMO15'),(NEXTVAL('id_seq'),'2019-10-29',35,2,'8.41','Glamorgan','Bauer','Tanek','1992-11-05','1',''),(NEXTVAL('id_seq'),'2019-10-19',50,1,'55.83','Somerset','Brown','Dahlia','1990-09-20','1','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-02-18',91,3,'83.27','Sutherland','Roth','Ebony','2005-01-01','0',''),(NEXTVAL('id_seq'),'2019-08-12',92,3,'27.15','Midlothian','Adkins','Hilel','1956-08-14','0',''),(NEXTVAL('id_seq'),'2019-06-27',17,4,'21.64','West Lothian','Harrington','Drew','1937-01-28','1',''),(NEXTVAL('id_seq'),'2019-08-15',79,4,'53.52','Northamptonshire','Wiggins','Wynter','1967-03-05','1','PROMO10');"
cmdFive = "INSERT INTO 'orders' VALUES (NEXTVAL('id_seq'),'2019-06-05',15,4,'60.47','Merionethshire','Hobbs','Wyoming','1934-07-24','0','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-05-24',19,1,'38.96','Shetland','Bravo','Sid','1966-11-01','1','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-01-07',76,4,'13.98','Bedfordshire','Valentine','Marah','1968-01-20','1',''),(NEXTVAL('id_seq'),'2019-11-13',21,4,'18.22','Hertfordshire','Davenport','Steven','1961-01-05','1',''),(NEXTVAL('id_seq'),'2019-05-13',16,2,'79.52','Stirlingshire','Mason','Dai','1970-07-19','1','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-12-05',26,4,'37.66','Cambridgeshire','Jefferson','Kylie','1955-03-27','1',''),(NEXTVAL('id_seq'),'2019-03-02',75,2,'9.46','Hertfordshire','Flores','Amber','1961-06-13','1',''),(NEXTVAL('id_seq'),'2019-12-06',98,4,'39.23','Roxburghshire','Burnett','Grace','2004-10-28','0','PROMO10'),(NEXTVAL('id_seq'),'2019-12-14',56,5,'40.41','Devon','Petersen','Nevada','1947-07-28','0',''),(NEXTVAL('id_seq'),'2019-01-10',37,4,'7.67','Derbyshire','Suarez','Mechelle','1934-06-16','1','');"
cmdSix = "INSERT INTO 'orders' VALUES (NEXTVAL('id_seq'),'2019-01-18',26,1,'66.98','Hampshire','Ramos','Adele','1995-04-04','0','PROMO10'),(NEXTVAL('id_seq'),'2019-11-11',70,3,'20.77','Cambridgeshire','Bruno','Jordan','1997-06-05','0','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-06-15',1,3,'28.61','Angus','Carr','Joelle','1999-04-09','1',''),(NEXTVAL('id_seq'),'2019-08-16',58,2,'41.86','Somerset','Brown','Dahlia','1990-09-20','1',''),(NEXTVAL('id_seq'),'2019-12-02',80,2,'35.19','Somerset','Taylor','Teagan','1962-02-08','0',''),(NEXTVAL('id_seq'),'2019-08-21',91,4,'72.23','Cheshire','Olsen','Winifred','1971-07-14','1',''),(NEXTVAL('id_seq'),'2019-07-04',99,2,'45.94','Worcestershire','Melendez','Brendan','1940-01-01','1',''),(NEXTVAL('id_seq'),'2019-06-11',3,2,'34.79','Brecknockshire','Vasquez','Abdul','1939-05-02','1','PROMO10'),(NEXTVAL('id_seq'),'2019-09-21',100,2,'60.13','Monmouthshire','Mclaughlin','Veda','1953-07-10','0',''),(NEXTVAL('id_seq'),'2019-09-13',99,1,'82.91','Denbighshire','Singleton','Rafael','1950-09-30','0','SUMMER MADNESS');"
cmdSeven = "INSERT INTO 'orders' VALUES (NEXTVAL('id_seq'),'2019-08-16',27,3,'2.84','Cardiganshire','Reilly','Aileen','1950-12-31','1','PROMO15'),(NEXTVAL('id_seq'),'2019-04-11',71,5,'55.81','Northamptonshire','Cross','Xenos','1960-03-25','0',''),(NEXTVAL('id_seq'),'2019-08-26',85,2,'17.62','East Lothian','Carpenter','Brielle','1979-04-28','0','PROMO10'),(NEXTVAL('id_seq'),'2019-05-31',14,5,'73.22','Shetland','Bravo','Sid','1966-11-01','1','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-04-20',42,3,'23.98','Cumberland','Byers','Xerxes','1950-08-26','0',''),(NEXTVAL('id_seq'),'2019-09-07',6,3,'6.75','Kent','Barlow','Harriet','1990-12-15','0',''),(NEXTVAL('id_seq'),'2019-08-26',83,3,'72.03','Yorkshire','Velez','Martha','1996-04-09','0',''),(NEXTVAL('id_seq'),'2019-08-22',17,4,'68.35','Lincolnshire','Valdez','Gil','1964-04-05','1','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-05-19',74,2,'93.23','Wigtownshire','Ferguson','Teagan','1962-04-09','1','PROMO10'),(NEXTVAL('id_seq'),'2019-01-12',68,5,'64.75','Sussex','Mann','Simon','1995-08-27','0','PROMO10');"
cmdEight = "INSERT INTO 'orders' VALUES (NEXTVAL('id_seq'),'2019-08-17',22,3,'88.35','Berkshire','Bender','Declan','1987-04-11','0',''),(NEXTVAL('id_seq'),'2019-04-01',38,3,'38.74','Yorkshire','Schmidt','Tatyana','1936-06-23','0','PROMO15'),(NEXTVAL('id_seq'),'2019-05-04',54,4,'90.49','Hertfordshire','Davenport','Steven','1961-01-05','1','PROMO15'),(NEXTVAL('id_seq'),'2019-06-18',55,4,'57.00','Cambridgeshire','Bruno','Jordan','1997-06-05','0','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-12-17',79,4,'85.81','Hertfordshire','Davenport','Steven','1961-01-05','1',''),(NEXTVAL('id_seq'),'2019-03-12',88,5,'92.22','Ross-shire','Hayden','Amena','1998-04-27','0',''),(NEXTVAL('id_seq'),'2019-08-12',47,5,'32.87','Banffshire','Thomas','Jermaine','1948-05-26','1','PROMO15'),(NEXTVAL('id_seq'),'2019-04-18',62,4,'12.97','Banffshire','Benjamin','Leo','1976-05-11','0','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-09-01',82,2,'47.85','Kincardineshire','Russo','Aristotle','1944-07-19','0',''),(NEXTVAL('id_seq'),'2019-08-18',53,3,'39.49','Suffolk','Davenport','Tyrone','1989-03-16','1','PROMO15');"
cmdNine = "INSERT INTO 'orders' VALUES (NEXTVAL('id_seq'),'2019-07-02',16,1,'55.84','Radnorshire','Reid','Gil','1984-06-17','0','PROMO15'),(NEXTVAL('id_seq'),'2019-01-15',15,4,'87.08','Northumberland','Cohen','Aaron','1953-04-14','0','PROMO15'),(NEXTVAL('id_seq'),'2019-01-27',36,4,'56.84','Suffolk','Davenport','Tyrone','1989-03-16','1','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-10-29',54,4,'8.85','Sussex','Moon','Keefe','1996-10-07','1',''),(NEXTVAL('id_seq'),'2019-09-02',26,4,'13.36','Northamptonshire','Nolan','Nora','1999-03-01','1','SUMMER MADNESS'),(NEXTVAL('id_seq'),'2019-08-11',86,5,'43.22','Monmouthshire','Flynn','Timon','1953-01-06','0',''),(NEXTVAL('id_seq'),'2019-02-14',8,5,'85.17','Montgomeryshire','Wright','Belle','1958-09-02','0',''),(NEXTVAL('id_seq'),'2019-09-13',46,2,'40.40','Staffordshire','Lynn','Rachel','1995-01-11','1',''),(NEXTVAL('id_seq'),'2019-12-16',9,1,'34.00','Kirkcudbrightshire','Frederick','Kane','2002-11-22','0',''),(NEXTVAL('id_seq'),'2019-12-08',6,2,'19.37','Anglesey','Cannon','Sydnee','1981-12-09','1','');"
cmdTen = "INSERT INTO 'orders' VALUES (NEXTVAL('id_seq'),'2019-04-24',7,5,'43.24','Cambridgeshire','Bruno','Jordan','1997-06-05','0',''),(NEXTVAL('id_seq'),'2019-06-24',83,4,'43.10','Huntingdonshire','Guthrie','Amela','1968-06-09','1','PROMO10'),(NEXTVAL('id_seq'),'2019-05-22',72,1,'28.35','Radnorshire','Shelton','Wing','1985-11-24','1',''),(NEXTVAL('id_seq'),'2019-08-27',4,2,'80.91','Kincardineshire','Suarez','Bradley','1960-05-12','1','PROMO15'),(NEXTVAL('id_seq'),'2019-12-16',50,4,'44.93','Suffolk','Carr','Miriam','1979-07-03','0',''),(NEXTVAL('id_seq'),'2019-09-03',28,4,'52.90','Cambridgeshire','Evans','Mallory','1954-04-10','1',''),(NEXTVAL('id_seq'),'2019-08-05',52,3,'62.49','Yorkshire','Riggs','Veronica','2004-02-02','0','PROMO15'),(NEXTVAL('id_seq'),'2019-05-03',40,1,'44.26','Derbyshire','Duncan','Flavia','1960-03-27','0','PROMO15'),(NEXTVAL('id_seq'),'2019-10-29',18,1,'56.37','Brecknockshire','Mejia','Bert','1969-01-12','1','PROMO10'),(NEXTVAL('id_seq'),'2019-02-28',98,5,'93.49','Dumfriesshire','Larsen','Genevieve','1981-09-20','0','');"

cmds = [cmdOne, cmdTwo, cmdThree, cmdFour, cmdFive, cmdSix, cmdSeven, cmdEight, cmdNine, cmdTen]

for cmd in cmds:
  conn.execute(cmd)

Syntax-wise this is the same as before. However, because we are managing our SQL queries as Python variables we can use time-savers such as the for loop that executes them all one-by-one.

Again, let's check this has worked by running some typical queries:

In [10]:
conn.execute("SELECT COUNT(*) FROM 'orders'").fetchall()

[(100,)]

In [11]:
conn.execute("SELECT * FROM 'orders' WHERE category_id=1;").fetchall()

[(4,
  '2019-01-26',
  1,
  1,
  '46.51',
  'Inverness-shire',
  'Jensen',
  'Jillian',
  '1948-05-07',
  '0',
  'PROMO15'),
 (19,
  '2019-10-29',
  44,
  1,
  '91.95',
  'Wiltshire',
  'Lang',
  'Hunter',
  '1981-12-10',
  '1',
  ''),
 (20,
  '2019-02-18',
  62,
  1,
  '56.23',
  'Caithness',
  'Mueller',
  'Kyla',
  '1990-04-21',
  '0',
  'PROMO10'),
 (25,
  '2019-09-24',
  13,
  1,
  '80.68',
  'Angus',
  'Simpson',
  'Nicole',
  '1973-07-11',
  '0',
  ''),
 (34,
  '2019-05-21',
  2,
  1,
  '64.61',
  'Caithness',
  'Bowman',
  'Naomi',
  '1979-04-16',
  '1',
  'PROMO15'),
 (36,
  '2019-10-19',
  50,
  1,
  '55.83',
  'Somerset',
  'Brown',
  'Dahlia',
  '1990-09-20',
  '1',
  'SUMMER MADNESS'),
 (42,
  '2019-05-24',
  19,
  1,
  '38.96',
  'Shetland',
  'Bravo',
  'Sid',
  '1966-11-01',
  '1',
  'SUMMER MADNESS'),
 (51,
  '2019-01-18',
  26,
  1,
  '66.98',
  'Hampshire',
  'Ramos',
  'Adele',
  '1995-04-04',
  '0',
  'PROMO10'),
 (60,
  '2019-09-13',
  99,
  1,
  '82.91',
  'Denbi

In [12]:
conn.execute("SELECT * FROM 'orders' WHERE county LIKE 'D%';").fetchall()

[(13,
  '2019-07-18',
  61,
  4,
  '53.26',
  'Derbyshire',
  'Dillon',
  'Amethyst',
  '1961-06-13',
  '1',
  ''),
 (16,
  '2019-05-16',
  32,
  3,
  '76.93',
  'Denbighshire',
  'Cleveland',
  'Amethyst',
  '1999-03-31',
  '1',
  ''),
 (18,
  '2019-08-19',
  44,
  5,
  '21.04',
  'Dunbartonshire',
  'Branch',
  'Brenda',
  '1983-03-10',
  '1',
  ''),
 (30,
  '2019-06-22',
  17,
  4,
  '50.22',
  'Denbighshire',
  'Dickerson',
  'Evangeline',
  '1972-08-16',
  '0',
  'PROMO15'),
 (49,
  '2019-12-14',
  56,
  5,
  '40.41',
  'Devon',
  'Petersen',
  'Nevada',
  '1947-07-28',
  '0',
  ''),
 (50,
  '2019-01-10',
  37,
  4,
  '7.67',
  'Derbyshire',
  'Suarez',
  'Mechelle',
  '1934-06-16',
  '1',
  ''),
 (60,
  '2019-09-13',
  99,
  1,
  '82.91',
  'Denbighshire',
  'Singleton',
  'Rafael',
  '1950-09-30',
  '0',
  'SUMMER MADNESS'),
 (98,
  '2019-05-03',
  40,
  1,
  '44.26',
  'Derbyshire',
  'Duncan',
  'Flavia',
  '1960-03-27',
  '0',
  'PROMO15'),
 (100,
  '2019-02-28',
  98,
  5,
 

Our syntax is nearly identical to "normal" SQL in databases (although we do not use punctuation or quotation marks around fields). All seems to be working so let's create the remaining tables.

In [13]:
conn.execute("CREATE SEQUENCE cat_id_seq START 1;")

conn.execute("CREATE TABLE 'categories' (id INTEGER NOT NULL PRIMARY KEY default NEXTVAL('cat_id_seq'), category_name VARCHAR default NULL);")

conn.execute("INSERT INTO 'categories' VALUES (NEXTVAL('cat_id_seq'),'Party'),(NEXTVAL('cat_id_seq'),'Gadgetts'),(NEXTVAL('cat_id_seq'),'Costume'),(NEXTVAL('cat_id_seq'),'Celebration'),(NEXTVAL('cat_id_seq'),'Joke');")

conn.execute("CREATE TABLE 'promotions' (code VARCHAR NOT NULL PRIMARY KEY, discount DOUBLE DEFAULT NULL);")

conn.execute("INSERT INTO 'promotions' VALUES ('PROMO10', '10'),('PROMO15', '15'),('SUMMER MADNESS','35');")

With multiple tables we can now execute joins and make use of the relational structure:

In [14]:
conn.execute("SELECT o.customer_firstname, o.customer_surname, c.category_name FROM orders o LEFT JOIN categories c ON (o.category_id = c.id);").fetchall()

[('Sid', 'Bravo', 'Gadgetts'),
 ('Elvis', 'Patrick', 'Gadgetts'),
 ('Tyrone', 'Davenport', 'Gadgetts'),
 ('Jillian', 'Jensen', 'Party'),
 ('Mona', 'Sexton', 'Gadgetts'),
 ('Xandra', 'Hooper', 'Costume'),
 ('Velma', 'Leonard', 'Costume'),
 ('Rana', 'Pope', 'Gadgetts'),
 ('Griffin', 'Velez', 'Costume'),
 ('Hadassah', 'Anderson', 'Joke'),
 ('Quinn', 'Barr', 'Celebration'),
 ('Grace', 'Carr', 'Costume'),
 ('Amethyst', 'Dillon', 'Celebration'),
 ('Fritz', 'Lee', 'Joke'),
 ('Caesar', 'Hunt', 'Celebration'),
 ('Amethyst', 'Cleveland', 'Costume'),
 ('Clare', 'Greene', 'Gadgetts'),
 ('Brenda', 'Branch', 'Joke'),
 ('Hunter', 'Lang', 'Party'),
 ('Kyla', 'Mueller', 'Party'),
 ('Dahlia', 'Brown', 'Gadgetts'),
 ('Tyrone', 'Davenport', 'Joke'),
 ('Jordan', 'Bruno', 'Celebration'),
 ('Thor', 'Schneider', 'Costume'),
 ('Nicole', 'Simpson', 'Party'),
 ('Martin', 'Golden', 'Costume'),
 ('Fiona', 'Massey', 'Celebration'),
 ('Ashton', 'Jefferson', 'Celebration'),
 ('Hammett', 'Ingram', 'Gadgetts'),
 ('Evan

Here we have a simple _LEFT JOIN_ that combines customer name and product category by joining on the category id.

## Fake Data
When building a prototype, sometimes it can be time consuming to create synthetic data to populate our tables. This example uses the Python package _Faker_ to automate this for us:

In [15]:
!pip install faker

import random
import duckdb
import pandas as pd
from faker import Faker
import sys

fake = Faker()

def get_person():
  person = {}
  person['id'] = random.randrange(1000,9999999999999)
  person['first_name'] = fake.first_name()
  person['last_name'] = fake.last_name()
  person['email'] = fake.unique.ascii_email()
  person['company'] = fake.company()
  person['phone'] = fake.phone_number()
  return person

personlist = []
for x in range(10):
  personlist.append(get_person())

df = pd.DataFrame.from_dict(personlist)

conn.execute("CREATE TABLE persons AS SELECT * FROM df")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.7 MB/s eta 0:00:00


After installing and loading the package we go through the following steps:


1.   We create a function called _get\_person()_ which uses _Faker_ to create a range of variables about a fake person (e.g. name and email address). These are stored into a Python dictionary which is returned at the end;
2.   We use a for loop to create 10 fake people and save them in a list (_personlist_);
3. We create a [_Pandas_ dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (a popular data science tool - basically like storing the data as an Excel worksheet) to store our list;
4. We create a new DuckDB table directly from this dataframe.

We can see this is a fairly quick and painless process and shows us another reason why DuckDB has become popular in Python use-cases ... the ability to quickly and easily create and populate tables from within a Python script/process. Note we didn't even need to specify the datatypes of the fields, these are inferred directly from their corresponding Python values (e.g. string values stored as VARCHAR, int values as integers, etc.).

To conclude, we can quickly check the table with a query:



In [16]:
conn.execute("SELECT * FROM persons;").fetchall()

[(665844710662,
  'Brenda',
  'Rodriguez',
  'mhart@james.info',
  'Rivers-Coleman',
  '(630)731-3508'),
 (3010002433709,
  'Regina',
  'Anderson',
  'daniel10@yahoo.com',
  'Williams PLC',
  '(493)766-1759x01910'),
 (5982549428911,
  'Evan',
  'Murray',
  'richardcardenas@yahoo.com',
  'Hamilton, Shaw and Norton',
  '473-774-2353x01410'),
 (2652638526904,
  'Jack',
  'Ford',
  'stevenmartinez@yahoo.com',
  'Elliott, Hernandez and Dixon',
  '2778472323'),
 (7356164727263,
  'Angela',
  'Sandoval',
  'robertcastillo@yahoo.com',
  'Hill-Howard',
  '319-448-2531'),
 (2239672009845,
  'Eric',
  'Marquez',
  'lopezkevin@yahoo.com',
  'Sutton, Meadows and Mahoney',
  '590-710-1594'),
 (151097069544,
  'Sonia',
  'Martin',
  'stephaniemyers@hernandez.com',
  'Freeman and Sons',
  '001-970-759-0075x0472'),
 (8950444943540,
  'Elizabeth',
  'Mejia',
  'jlloyd@vargas.net',
  'Hall, Humphrey and Friedman',
  '(984)796-4105x1838'),
 (7294596735716,
  'Jonathan',
  'Townsend',
  'tatebrandi@king.co

## Creating a table from a CSV file
Our final example will show creating a table directly from a CSV file. If it is easier to create your data in Excel and then import, this is the easiest option.

To start, download the file _apples\_and\_oranges.csv_ from the Github. We can upload this to Colab by running the following:

In [17]:
from google.colab import files
upload = files.upload()

Saving apples_and_oranges.csv to apples_and_oranges.csv


Selecting the downloaded file from the file picker saves it into our Colab instance. With the file setup we can create a database table directly from it:

In [18]:
conn.execute("CREATE TABLE fruit AS SELECT * FROM apples_and_oranges.csv;")

We can check it's working with a simple SQL query to select two of the fields created:

In [19]:
conn.execute("SELECT fruit_name, mass FROM fruit;").fetchall()

[('apple', 192),
 ('apple', 180),
 ('apple', 176),
 ('apple', 178),
 ('apple', 172),
 ('apple', 166),
 ('apple', 172),
 ('apple', 154),
 ('apple', 164),
 ('apple', 152),
 ('apple', 156),
 ('apple', 156),
 ('apple', 168),
 ('apple', 162),
 ('apple', 162),
 ('apple', 160),
 ('apple', 156),
 ('apple', 140),
 ('apple', 170),
 ('orange', 342),
 ('orange', 356),
 ('orange', 362),
 ('orange', 204),
 ('orange', 140),
 ('orange', 160),
 ('orange', 158),
 ('orange', 210),
 ('orange', 164),
 ('orange', 190),
 ('orange', 142),
 ('orange', 150),
 ('orange', 160),
 ('orange', 154),
 ('orange', 158),
 ('orange', 144),
 ('orange', 154),
 ('orange', 180),
 ('orange', 154)]

As before, this gives us a really quick way to get data into our DB. Also as before, DuckDB has also inferred all the column headings and data types (we can see the _fruit\_name_ is represented as a string (VARCHAR) and _mass_ as an integer).